Car Pipeline:
Car Detection (YOLO) -> detect and classify driver and seatbelt (Custom) Not Done Yet -> license plate detection(YOLO) -> OCR (pytesseract)


In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip -q install ultralytics
    !pip -q install paddleocr[all]
    if not os.path.exists('license-plate-finetune-v1s.pt'):
        !wget https://huggingface.co/morsetechlab/yolov11-license-plate-detection/resolve/main/license-plate-finetune-v1s.pt



In [ ]:
from ultralytics import YOLO
from pathlib import Path
from paddleocr import PaddleOCR

license_plate_detection_model = YOLO('license-plate-finetune-v1s.pt')

car_detection_model = YOLO("yolo11n.pt")
ocr_en = PaddleOCR(use_angle_cls=True, lang='en')
    


In [ ]:
def detect_cars_and_plates(image_path: Path | str):
    # First, detect cars in the image
    car_results = car_detection_model.predict(source=image_path, classes=[2,5,7])  # COCO classes for car, bus, truck
    output_plates = []
    for car in car_results[0].boxes:
        # Extract car bounding box
        x1, y1, x2, y2 = map(int, car.xyxy[0])
        car_image = car_results[0].orig_img[y1:y2, x1:x2]
        
        plate_results = license_plate_detection_model.predict(source=car_image)
                
    return plate_results

In [ ]:
def run_ocr_on_plate(plate_results):
    ocr_results = []
    for plate in plate_results[0].boxes:
        x1, y1, x2, y2 = map(int, plate.xyxy[0])
        plate_image = plate_results[0].orig_img[y1:y2, x1:x2]
        
        ocr_result = ocr_en.predict(plate_image)
        ocr_results.append(ocr_result)
    return ocr_results
    

In [ ]:
def visualize_pipeline_results(image_path: Path | str):
    plate_results = detect_cars_and_plates(image_path)
    ocr_results = run_ocr_on_plate(plate_results)
    # show bounding boxes with ocr results
    plate_results[0].show(boxes=True, labels=[res[0][1][1] for res in ocr_results])
    

In [ ]:
if 'google.colab' in sys.modules:
    if "bmwalleng.png" not in os.listdir("."):
      !wget https://github.com/AFAskar/dlimageclass/blob/main/bmwalleng.png?raw=true
      !mv bmwalleng.png?raw=true bmwalleng.png


images = ['./bmwalleng.png']
for img_path in images:
    visualize_pipeline_results(img_path)
        
    

